In [ ]:
%load_ext autoreload

In [ ]:
from filemon.orchestrate import monitor_files, RestOrchestrator, RestPuller, IntCapturer
import requests

rest_reporting_host="192.168.120.119"
rest_reporting_port="5000"

rest_puller_host="192.168.120.119"
rest_puller_port="5000"
rest_puller_api="api/configs/"
rest_puller_update_period_s=1
storage = "/data/"

def get_config():
    get_url = "http://192.168.120.119:5000/api/config/"
    try:
        results = requests.get(get_url)
    except requests.exceptions.RequestException:
        return None
    if results.status_code==200:
        return results.json()
    else: 
        return None

def get_restids():
    global storage
    ids = []
    titles = []
    files = []
    cfg = get_config()
    for site in cfg['sites'].values():
        ids.append(site['dtn'])
        titles.append(site['dtn'] + " rtt: " + str(site['rtt']) + " ms")
        files.append(storage + site['dtn'])
    return ids,titles,files

rest_ids, titles, files = get_restids()   
 

ro = RestOrchestrator()
ro.init(rest_ids, titles, 
        y_factor=1.0/float(1024*1024),
        y_label="MB/s", y_lim=(0, 20),
        rest_reporting=False,
        hostname=rest_reporting_host, port=rest_reporting_port)

rp = RestPuller(rest_puller_host, rest_puller_port, rest_puller_api,
                ro.process_changes, update_period_s=rest_puller_update_period_s)
rp.start()

ic = IntCapturer()
ic.program_capture_stop([ro, rp])


In [ ]:
%%bash
curl -H "Content-Type: application/json" -d '{"dest":"/data/12"}' http://localhost:5000/start/12 -X PUT 
